## Instructor and responsemodel

In [ ]:
#imports
# built-in libraries
from typing import TypeVar, Literal, Any
from pydantic import BaseModel, create_model

# litellm libraries
import litellm
from litellm import completion
from litellm.types.utils import ModelResponse, Message

#instructor
from instructor import from_litellm, Mode

# misc libraries
from decouple import config
from pydantic import BaseModel, Field, create_model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams


setting up the watsonx credentials


In [ ]:
WX_API_KEY = config("WX_API_KEY")
WX_PROJECT_ID = config("WX_PROJECT_ID")
WX_API_URL = "https://us-south.ml.cloud.ibm.com"
WX_MODEL_ID = "*"

Initializing our instructor and creating our response model


In [ ]:

class BaseResponse(BaseModel):
    answer: str
ResponseType = TypeVar('ResponseType', bound=BaseModel)



class LLMCaller:

    def __init__(self, api_key: str, project_id: str, api_url: str, model_id: str, params: dict[str, Any]):
        self.api_key = WX_API_KEY
        self.project_id = WX_PROJECT_ID
        self.api_url = WX_API_URL
        self.model_id = WX_MODEL_ID
        self.params = params

        # Boilerplate: Configure LiteLLM to drop unsupported parameters for Watsonx.ai
        litellm.drop_params = True
        # Boilerplate: Create an Instructor client for pydantic-based interactions with the LLM
        self.client = from_litellm(completion, mode=Mode.JSON)

    def create_response_model(self, title: str, fields: dict) -> ResponseType:
        """ Dynamically creates a Pydantic response model for the LLM's output.
        Args:
            title (str): The name of the response model.
            fields (dict): A dictionary defining the fields of the response model.
                           Keys are field names, and values are tuples of (type, Field).

        Returns:
            ResponseType: A dynamically created Pydantic model class.
        """
        return create_model(title, **fields, __base__=BaseResponse)

    def invoke(self, prompt: str, response_model: ResponseType = BaseResponse, **kwargs) -> ResponseType:
        """ Sends a prompt to the LLM and retrieves a structured response.

        Args:
            prompt (str): The input prompt to send to the LLM.
            response_model (ResponseType): The Pydantic model to structure the LLM's response.
                                           Defaults to BaseResponse.
            **kwargs: Additional arguments to pass to the LLM client.

        Returns:
            ResponseType: The structured response from the LLM, parsed into the specified response model.
        """
        response = self.client.chat.completions.create(
            model=self.model_id,
            messages=[
                {
                    "role": "user",
                    "content": prompt + "\n\n" + f"Provide your answer as an object of {type(response_model)}",
                    # notice how we hardcode instructions on the responde model type for the llm
                    # so we don't have to repeat it in the prompt
                }
            ],
            project_id=self.project_id,
            apikey=self.api_key,
            api_base=self.api_url,
            response_model=response_model,
            **kwargs
        )
        return response